In [114]:
from fpylll import *
load('StatBKZ.py')
load('experiments.py')
load('framework/instance_gen.sage')
n = 256
m = 128
q = 3229
k = 2 #2/3/4
g = GF(q)(17)
sds = 1.22 #1.22/1/1
sde = 1.22
D_s = build_centered_binomial_law(3) #3/2/2
D_e = build_centered_binomial_law(3)

In [77]:
def bit_reverse(n):
    return int('{:08b}'.format(n)[::-1], 2)


def gen_M(): # Matrix for NTT2
    M = []
    for i in range(m):
        gg = g**(bit_reverse(i) + 1)
        mm = [int(gg**i) for i in range(m)]
        M.append(mm)
    M2 = []
    for i in range(m):
        v = []
        for j in range(m):
            v.append(M[i][j])
            v.append(0)
        M2.append(v)
        M2.append([0]+v[:-1])
    return matrix(GF(q), M2)

In [106]:
Ds=DiscreteGaussianDistributionIntegerSampler(sds)
De=DiscreteGaussianDistributionIntegerSampler(sde)
s=[vector([Ds() for _ in range(n)]) for _ in range(k)]
e=[vector([De() for _ in range(n)]) for _ in range(k)]
M = gen_M()

sh = []
for i in range(k):
    sh.append(M*s[i])

A = []
for i in range(k):
    a = []
    for j in range(k):
        aa = [[random.randint(0, q-1) for i in range(n)] for _ in range(n)]
        aa = matrix(GF(q), aa)
        a.append(aa)
    A.append(a)

b = []
for i in range(k):
    bb = A[i][0] * s[0]
    for j in range(1, k):
        bb += A[i][j] * s[j]
    b.append(bb + e[i])

A = block_matrix(A)
M = block_matrix([[M, zero_matrix(n), zero_matrix(n)],[zero_matrix(n), M, zero_matrix(n)], [zero_matrix(n), zero_matrix(n), M]])
s = vector(concatenate(s))
sh = vector(concatenate(sh))
e = vector(concatenate(e))
b = vector(concatenate(b))

In [87]:
def build_LWE_by_se2(A, M, MI, s, e, b, knowns, sample):

    zeros = n*k - knowns
    zero_index = random.sample(list(range(n*k)), zeros)
    sample_index = random.sample(list(range(n*k)), sample)
    MA = MI*A.T

    v = vector(GF(q), M * s)
    for i in zero_index:
        v[i] = 0

    t1 = v * MI
    t2 = v * MI * A.T - b

    a = []
    es = []
    t2s = []
    for i in zero_index:
        ma = []
        for j in sample_index:
            ma.append(MA[i][j])
        a.append(list(MI[i]) + ma)
        
    for j in sample_index:
        t2s.append(t2[j])
        es.append(-e[j])

    a = Matrix(GF(q), a).echelon_form().change_ring(ZZ)

    b1 = []
    for i in range(zeros):
        b1.append(list(a[i]) + [0])

    for i in range(zeros, n*k+sample):
        b1.append([0] * i + [q] + [0] * (n*k-i+sample))

    b1.append(list(t1) + t2s + [1])

    return matrix(ZZ, A), matrix(ZZ, b1), concatenate(list(s) + es, [1])


In [116]:
def initialize_from_NTT_instance(knowns, sample, dbdd_class, n, q, D_s, D_e, diag=False, verbosity=1):
    mu_s, s_s = average_variance(D_s)
    mu_e, s_e = average_variance(D_e)

    s = [vector([draw_from_distribution(D_s) for _ in range(n)]) for _ in range(k)]
    e = [vector([draw_from_distribution(D_s) for _ in range(n)]) for _ in range(k)]
    M = gen_M()

    A = []
    for i in range(k):
        a = []
        for j in range(k):
            aa = [[random.randint(0, q-1) for i in range(n)] for _ in range(n)]
            aa = matrix(GF(q), aa)
            a.append(aa)
        A.append(a)

    b = []
    for i in range(k):
        bb = A[i][0] * s[0]
        for j in range(1, k):
            bb += A[i][j] * s[j]
        b.append(bb + e[i])

    A = block_matrix(A)
    M = block_matrix([[M, zero_matrix(n)], [zero_matrix(n), M]])
    #M = block_matrix([[M, zero_matrix(n), zero_matrix(n)],[zero_matrix(n), M, zero_matrix(n)], [zero_matrix(n), zero_matrix(n), M]])
    s = vector(concatenate(s))
    e = vector(concatenate(e))
    b = vector(concatenate(b))
    MI = M.inverse().T
    
    A, B, u = build_LWE_by_se2(A, M, MI, s, e, b, knowns, sample)
    mu = vec((n*k) * [mu_s] + sample * [mu_e] + [1])
    S = diagonal_matrix((n*k) * [s_s] + sample * [s_e] + [0])
    return A, b, dbdd_class(B, S, mu, u, verbosity=verbosity, float_type="dd")


In [117]:
est1 = []
est2 = []
expe = []
trials = 1
do_exp = False
for knowns in range(32, 128, 32):
  for sample in range(0, 512, 16):
    b1 = 0
    b2 = 0
    x0 = 0
    wk = 0
    for _ in range(trials):
        A, b, dbdd = initialize_from_NTT_instance(knowns, sample, DBDD, n, q, D_s, D_e)
        #beta1, _ = dbdd.estimate_attack(probabilistic=True)
        beta2, _ = dbdd.estimate_attack(probabilistic=False)
        #b1 += beta1
        b2 += beta2
        if do_exp:
            x, y = dbdd.attack()
            x0 += x
            wk += 1
    #est1.append(b1/trials)
    est2.append(b2/trials)
    if do_exp:
        expe.append(x0/wk)
  print(knowns, est2)

       Attack Estimation      
 dim=513 	 δ=0.000000 	 β=9999.00  
  


KeyboardInterrupt: 